# Загрузка библиотек

Если какие-то из библиотек не установлены - используйте !pip install

Для работы на Kaggle требуется верифицированный по номеру телефона профиль (без него не сработает pip install (точнее, нельзя будет в настройках включить работу с интернетом) и не подключить графические ускорители). Если при попытке верификации по номеру выдает ошибку "this phone number can't be verified" - попробуйте обратиться в техподдержку Kaggle, они ответят в течении 1-2 дней и уберут ошибку при вводе того же номера (личный опыт).

На удаленных средах (Kaggle, Google Colab), для работы с данными/модификации преобразований изображений, графические ускорители можно не подключать, и работать в стандартной среде. В противном случае - перед запуском кода подлючите графические ускорители (в Kaggle: Session Options -> Accelerator -> GPU T4 x 2, в Google Colab - меню рядом со статусом ОЗУ/Диск -> Сменить среду выполнения -> Графический ускоритель T4).

In [ ]:
!pip install colorama

In [ ]:
!pip install segmentation-models-pytorch

In [ ]:
!pip install GPUtil

In [ ]:
import numpy as np
import pandas as pd

import random
import glob
import os, shutil
from tqdm import tqdm
tqdm.pandas()
import time
import copy
import joblib
from collections import defaultdict
import gc
from IPython import display as ipd

# visualization
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# Sklearn
from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold
from sklearn.model_selection import train_test_split

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

from joblib import Parallel, delayed

# For colored terminal text
from colorama import Fore, Back, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import sys
import h5py
from sklearn.preprocessing import StandardScaler

#Модели машинного обучения
import segmentation_models_pytorch as smp

#Утилиты для работы с GPU
import GPUtil

# Загрузка данных

Перед работой с данными, их нужно загрузить на компьютер/удаленную среду.

Для загрузки на компьютер, используйте скрипт для загрузки (находится в другом jupyter notebook).

Для загрузки на Google Colab с компьютера ни в коем случае не используйте внутрисессионное хранилище - данные пропадут при смене среды (а грузится они будут очень долго), загрузите данные на Google Drive в архиве .zip (во первых так данные будут меньше, а во вторых процедура "загрузка .zip с Google Drive в Google Colab -> распаковка .zip в Google Colab" быстрее процедуры "загрузи все несжатые данные с Google Drive", в третьих в стандартный бесплатный объем Google Drive несжатые данные не влезут).

Для загрузки на Kaggle с компьютера испольйте в меню справа Input -> Upload. Советую перед загрузкой положить данные в .zip архив, это уменьшит время загрузки данных. Kaggle самостоятельно распакует архив и создаст датасет для работы.

В данной реализации сегментации, используются только Cor снимки, поэтому, при желании, можно не загружать другие типы снимков (это существенно уменьшит вес загружаеммых данных).

In [ ]:
#Преобразует tif файлы в массив масок и снимков
def load_tif(x_path, y_path,count_dict, step=4):
    images = []
    masks = []
    image = Image.open(x_path)
    mask = Image.open(y_path)
    i = 0
    cnt = 0

    while True:
        try:
            mask.seek(i)
            mask_array = np.array(mask)
            image.seek(i)
            image_array = np.array(image)
            if sum(sum(mask_array)) > 0: #Проверка, что маска не полностью черная

                #Если маска не из 0 и 1, а из, например, 0 и 255, меняем 255 на 1.
                mask_array[mask_array > 1] = 1

                masks.append(mask_array)
                images.append(image_array)
            cnt+=1

            i += 1
        except EOFError:
            break

    if 'T1' in x_path:
        count_dict['scan'].append('T1')
    elif 'T2' in x_path:
        count_dict['scan'].append('T2')
    count_dict['count'].append(cnt)

    return images, masks

In [ ]:
#Достаем изображения и маски из многослойного tif файла
def get_images(x_pathes, y_pathes):

    images =[]
    masks = []
    count_dict = {'scan':[],'count':[],}
    for x_path, y_path in zip(x_pathes, y_pathes):

        images_sample,mask_sample = load_tif(x_path, y_path,count_dict)
        for im,ms in zip(images_sample,mask_sample):

            images.append(im)
            masks.append(ms)

    print(len(count_dict['count']))

    return images, masks ,count_dict

In [ ]:
#Обходит директорию и сохранятет пути пациентов, вычленяет проекции Cor T1 и Cor T2 и маски
def get_pathes(path):
    x_pathes_all = []
    y_pathes_all = []
    for patient in os.listdir(path):
        x_pathes = []
        y_pathes = []

        for ID_s in os.listdir(path + '/'+ patient ):
            if 'ID' in ID_s:
                msk_t1 = 0
                msk_t2 = 0
                for tif_name in os.listdir(path + '/'+ patient + '/'+ID_s):

                    if 'Cor' in tif_name:
                        if 'T1' in tif_name:

                            if  'mask' not in tif_name.lower():
                                x_pathes.append(path + '/'+ patient + '/' + ID_s + '/'+ tif_name)

                            elif 'mask' in tif_name.lower():
                                msk_t1 = 1
                                y_pathes.append(path + '/'+ patient + '/'+ID_s + '/' + tif_name)

                        elif 'T2' in tif_name:

                            if 'mask' not in tif_name.lower():
                                x_pathes.append(path + '/'+ patient + '/'+ID_s + '/' + tif_name)

                            elif 'mask' in tif_name.lower():
                                msk_t2 = 1
                                y_pathes.append(path + '/'+ patient + '/'+ID_s + '/' + tif_name)

                if msk_t1==0:
                    x_pathes.pop()
                if msk_t2==0:
                    x_pathes.pop()

        x_pathes_all.append(x_pathes)
        y_pathes_all.append(y_pathes)

    return x_pathes_all, y_pathes_all

In [ ]:
def flatten(xss): # для развертки и однородности списков

    return [x for xs in xss for x in xs]

Указываем путь, где хранится папка с данными. Для корректной работы, данные должны быть в таком же формате, как формирует скрипт для скачки данных. Данный формат представляет собой папку, где данные каждого пациента хранятся в папках "ID_{номер_пациента}", в каждой такой папке хранится папка "ID {номер пациента}" с .tif изображениями, а также файл labels.txt с категориями Knosp. В текущей реализации сегментации, файлы labels.txt, а также все не коронарные снимки (без Cor в названии) не используются, поэтому, при работе с удаленной средой (Kaggle/Google colab) их можно не загружать (это в несколько раз уменьшит вес загружаемого файла с данными)

In [ ]:
#Указываем путь до папки с данными. Формат данных - как формирует скрипт загрузки датасета

#Примеры пути для разных сред

#При хранении на Google Drive при работе с Google Colab (в данном примере - данные не сжаты в .zip архиве)
#from google.colab import drive
#drive.mount('/content/drive')
#x_pth, y_pth = get_pathes('/content/drive/MyDrive/data_pituitary_test_light')

#Формат ссылки при хранении на Kaggle
x_pth ,y_pth = get_pathes('/kaggle/input/data-22-02-25-cls108-seg72')

#Формат ссылки, при хранении на локальном компьютере
#x_pth, y_pth = get_pathes('C:/Users/12345654321/data_pituitary_18_01_25')

In [ ]:
x_pth ,y_pth = [flatten(i) for i in [x_pth ,y_pth]]

In [ ]:
im,ms,cnt= get_images(x_pth ,y_pth)

In [ ]:
#Разбиваем данные на train и validation, при желании можно поменять test_size.

x_pth_train,x_pth_val,y_pth_train,y_pth_val = train_test_split(x_pth , y_pth, test_size=0.14)

In [ ]:
#Достает изображения и маски, преобразуя в массивы.

#Первая цифра output - кол-во tif файлов в test датасете (по 2 файла (Cor T1 и Cor T2) на пациента, не включая маски)
#Вторая цифра output - кол-во tif файлов в train датасете (по 2 файла (Cor T1 и Cor T2) на пациента, не включая маски)

x_val,y_val,_ = get_images(x_pth_val ,y_pth_val)

x_train,y_train,_= get_images(x_pth_train ,y_pth_train)

# Преобразование данных

In [ ]:
class BuildDataset(torch.utils.data.Dataset): # загружает в оперативную память

    def __init__(self, X,y, label=True, transforms=None):
        self.label      = label
        self.img_paths  = X
        self.msk_paths  = y
        self.transforms = transforms

    def __len__(self):

        return len(self.img_paths)

    def __getitem__(self, index):

        if self.label == True:

            img = self.img_paths[index]
            msk = self.msk_paths[index]

            if self.transforms:

                data = self.transforms(image=np.array(img/255., dtype=np.float32)  , mask=np.array(msk, dtype=np.float32))
                img  = data['image']
                msk  = data['mask']

            return torch.tensor([img]), torch.tensor([msk])

        else:

            img = self.img_paths[index]
            if self.transforms:
                data = self.transforms(image=np.array(img/255., dtype=np.float32))
                img  = data['image']
            return torch.tensor([img])

Здесь происходит преобразование данных с изображений (их обрезка, нормализация итд). Можно попробовать поизменять тут параметры для улучшения качества обучения. Будьте внимательны - не все преобразования можно не дублировать в valid. Повороты/dropout части картинки для улучшения качества обучения - норм, существенная обрезка изображения/изменение яркостей пикселей - не норм, дублируйте в valid.

In [ ]:
data_transforms = {
    "train": A.Compose([
        A.augmentations.crops.transforms.CenterCrop(256,256),
        A.Resize(224,224, interpolation=cv2.INTER_NEAREST),
        A.CoarseDropout(max_holes=8, max_height=224//20, max_width=224//20,
                         min_holes=5, fill_value=0, mask_fill_value=0, p=0.5),


        A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.05, rotate_limit=10, p=0.5),
        A.OneOf([
            A.GridDistortion(num_steps=5, distort_limit=0.05, p=1.0),
            # A.OpticalDistortion(distort_limit=0.05, shift_limit=0.05, p=1.0),
            A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)
        ], p=0.25),
        # A.ColorJitter(brightness=0, contrast=0.0002, saturation=0, hue=0.2, always_apply=False, p=0.5),
        # A.CoarseDropout(max_holes=8, max_height=224//20, max_width=224//20,
        #                  min_holes=5, fill_value=0, mask_fill_value=0, p=0.5),
        A.augmentations.Normalize(mean=(0.485, ), std=(0.229, )),
        ], p=1.0),

    "valid": A.Compose([
        A.augmentations.crops.transforms.CenterCrop(256,256),
        A.Resize(224,224, interpolation=cv2.INTER_NEAREST),
        A.augmentations.Normalize(mean=(0.485, ), std=(0.229, )),
        ], p=1.0)
}

Подготовка данных для загрузки в модель. Здесь можно поменять batch_size, это может повлиять на обучение.

In [ ]:
train_dataset = BuildDataset(x_train,y_train, transforms=data_transforms['train'])
valid_dataset = BuildDataset(x_val,y_val, transforms=data_transforms['valid'])

train_batch_size = 16
valid_batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=valid_batch_size, shuffle=False)

Здесь происходит визуализация преобразованных данных. Используйте чтобы примерно оценить, что произошло с данными после преобразования/найти ошибки в коде.

In [ ]:
def plot_batch(imgs, msks, size=3):
    plt.figure(figsize=(5*5, 5))
    for idx in range(size):
        plt.subplot(1, size, idx+1)
        img = imgs[idx,].permute((1, 2, 0)).numpy()*255.0
        msk = msks[idx,].permute((1, 2, 0)).numpy()
        show_img(img, msk)
    plt.tight_layout()
    plt.show()

In [ ]:
def show_img(img, mask=None):

    plt.imshow(img, cmap='bone')

    plt.imshow(mask, alpha=0.5)
    handles = [Rectangle((0,0),1,1, color=_c) for _c in [(0.667,0.0,0.0)]]
    labels = ["Adenoma"]
    plt.legend(handles,labels)
    plt.axis('off')

In [ ]:
#Изображение после train преобразования + маска.

imgs, msks = next(iter(train_loader))
plot_batch(imgs, msks, size=5)

In [ ]:
#Изображение после train преобразования

plot_batch(imgs, imgs, size=5)

In [ ]:
#Маска

plot_batch(msks, msks, size=5)

# Модель

In [ ]:
#Принудительно включаем garbage collector (чтобы улучшить производительность обучения)

gc.collect()

Архитектура модели. Используются реализации из библиотеки segmentation_models_pytorch. Саму модель/ее параметры (encoder_name) можно поменять.

In [ ]:
def build_model():
    model = smp.UnetPlusPlus(
        encoder_name='efficientnet-b1' ,
                  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        encoder_weights='imagenet',# use `imagenet` pre-trained weights for encoder initialization
        in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=1   ,     # model output channels (number of classes in your dataset)
        activation=None,
    )
    model.to('cuda')
    return model

def load_model(path):
    model = build_model()
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

# Loss

Здесь настраивается Loss (функция criterion), также есть фунции ручного расчета dice_coef и iou_coef.

In [ ]:
JaccardLoss = smp.losses.JaccardLoss(mode='binary')
DiceLoss    = smp.losses.DiceLoss(mode='binary')
BCELoss     = smp.losses.SoftBCEWithLogitsLoss()
LovaszLoss  = smp.losses.LovaszLoss(mode='binary', per_image=False)
TverskyLoss = smp.losses.TverskyLoss(mode='binary', log_loss=False)

def dice_coef(y_true, y_pred, thr=0.5, dim=(2,3), epsilon=0.001):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum(dim=dim)
    den = y_true.sum(dim=dim) + y_pred.sum(dim=dim)
    dice = ((2*inter+epsilon)/(den+epsilon)).mean(dim=(1,0))
    return dice

def iou_coef(y_true, y_pred, thr=0.5, dim=(2,3), epsilon=0.001):
    y_true = y_true.to(torch.float32)
    y_pred = (y_pred>thr).to(torch.float32)
    inter = (y_true*y_pred).sum(dim=dim)
    union = (y_true + y_pred - y_true*y_pred).sum(dim=dim)
    iou = ((inter+epsilon)/(union+epsilon)).mean(dim=(1,0))
    return iou

def criterion(y_pred, y_true):
    return BCELoss(y_pred, y_true) + TverskyLoss(y_pred, y_true)

# Обучение

In [ ]:
#Обучение одной эпохи

def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    scaler = amp.GradScaler()
    train_scores = []
    dataset_size = 0
    running_loss = 0.0

    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Train ')
    for step, (images, masks) in pbar:
        images = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)

        batch_size = images.size(0)

        with amp.autocast(enabled=True):
            y_pred = model(images)

            loss   = criterion(y_pred, masks)
            if (loss < 0):
                print("ALARM")
                print(loss)
                print("#######")

            loss   = loss / CFG.n_accumulate

        scaler.scale(loss).backward()

        if (step + 1) % CFG.n_accumulate == 0:
            scaler.step(optimizer)
            scaler.update()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()

        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        y_pred = (nn.Sigmoid()(y_pred)).double()
        train_dice = dice_coef(masks, y_pred).cpu().detach().numpy()
        vtrain_jaccard = iou_coef(masks, y_pred).cpu().detach().numpy()
        train_scores.append([train_dice, vtrain_jaccard])

        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_mem=f'{mem:0.2f} GB')

    train_scores  = np.mean(train_scores, axis=0)
    torch.cuda.empty_cache()
    gc.collect()

    return epoch_loss,train_scores

In [ ]:
#Валидация одной эпохи

@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0.0

    val_scores = []

    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid ')
    for step, (images, masks) in pbar:
        images  = images.to(device, dtype=torch.float)
        masks   = masks.to(device, dtype=torch.float)

        batch_size = images.size(0)

        y_pred  = model(images)
        loss    = criterion(y_pred, masks)

        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        y_pred = (nn.Sigmoid()(y_pred)).double()
        val_dice = dice_coef(masks, y_pred).cpu().detach().numpy()
        val_jaccard = iou_coef(masks, y_pred).cpu().detach().numpy()
        val_scores.append([val_dice, val_jaccard])

        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(valid_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_memory=f'{mem:0.2f} GB')


    val_scores  = np.mean(val_scores, axis=0)
    torch.cuda.empty_cache()
    gc.collect()

    return epoch_loss, val_scores

In [ ]:
#Процесс обучения

def run_training(model, optimizer, scheduler, device, num_epochs):
    # To automatically log gradients
    # wandb.watch(model, log_freq=100)

    if torch.cuda.is_available():
        print("cuda: {}\n".format(torch.cuda.get_device_name()))

    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_dice      = -np.inf
    best_epoch     = -1
    history = defaultdict(list)

    for epoch in range(1, num_epochs + 1):
        gc.collect()
        print(f'Epoch {epoch}/{num_epochs}', end='')
        train_loss,train_scores = train_one_epoch(model, optimizer, scheduler,
                                           dataloader=train_loader,
                                           device=CFG.device, epoch=epoch)
        train_dice,train_jaccard = train_scores

        print(f'Train Dice: {train_dice:0.4f} | Train Jaccard: {train_jaccard:0.4f}')

        val_loss, val_scores = valid_one_epoch(model, valid_loader,
                                                 device=CFG.device,
                                                 epoch=epoch)
        val_dice, val_jaccard = val_scores

        history['Train Loss'].append(train_loss)
        history['Train Dice'].append(train_dice)
        history['Train Jaccard'].append(train_jaccard)
        history['Valid Loss'].append(val_loss)
        history['Valid Dice'].append(val_dice)
        history['Valid Jaccard'].append(val_jaccard)

        # Log the metrics
        # wandb.log({"Train Loss": train_loss,
        #            "Valid Loss": val_loss,
        #            "Valid Dice": val_dice,
        #            "Valid Jaccard": val_jaccard,
        #            "LR":scheduler.get_last_lr()[0]})

        print(f'Valid Dice: {val_dice:0.4f} | Valid Jaccard: {val_jaccard:0.4f}')

        # deep copy the model
        if val_dice > best_dice:
            print(f"{c_}Valid Score Improved ({best_dice:0.4f} ---> {val_dice:0.4f})")
            best_dice    = val_dice
            best_jaccard = val_jaccard
            best_epoch   = epoch
            run.summary["Best Dice"]    = best_dice
            run.summary["Best Jaccard"] = best_jaccard
            run.summary["Best Epoch"]   = best_epoch
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"best_epoch-{fold:02d}.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            # wandb.save(PATH)
            print(f"Model Saved{sr_}")

        last_model_wts = copy.deepcopy(model.state_dict())
        PATH = f"last_epoch-{fold:02d}.bin"
        torch.save(model.state_dict(), PATH)

        print(); print()

    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Score: {:.4f}".format(best_jaccard))

    # load best model weights
    model.load_state_dict(best_model_wts)

    return model, history

Параметры модели.

Для изменения числа эпох, меняйте поле epochs. Для тестовых прогонов хватает 50-70 эпох. Для финального, в предыдущих работах используется 200 эпох.

Для изменения параметров планировщика, редактируйте поля scheduler, T_0, T_max, min_lr.

Начальный lr (или lr без планировщика) - поле lr.

In [ ]:
class CFG:
    seed          = 42
    debug         = False # set debug=False for Full Training
    exp_name      = 'Baselinev2'
    comment       = 'unetpp-efficientnet_b1-224x224-aug2-split2'
    model_name    = 'UnetPlusPlus'
    backbone      = 'efficientnet-b1'
    train_bs      = 128
    valid_bs      = train_bs*2
    img_size      = [224, 224]
    epochs        = 70
    lr            = 1e-4
    scheduler     = 'CosineAnnealingLR'
    min_lr        = 1e-6
    T_max         = int(30000/train_bs*epochs)+50
    T_0           = 25
    warmup_epochs = 0
    wd            = 1e-6
    n_accumulate  = max(1, 32//train_bs)
    n_fold        = 5
    num_classes   = 3
    device        = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Настройки параметров планировщика изменения lr со временем

In [ ]:
def fetch_scheduler(optimizer):
    if CFG.scheduler == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CFG.T_max,
                                                   eta_min=CFG.min_lr)
    elif CFG.scheduler == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CFG.T_0,
                                                             eta_min=CFG.min_lr)
    elif CFG.scheduler == 'ReduceLROnPlateau':
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,
                                                   mode='min',
                                                   factor=0.1,
                                                   patience=7,
                                                   threshold=0.0001,
                                                   min_lr=CFG.min_lr,)
    elif CFG.scheduer == 'ExponentialLR':
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.85)
    elif CFG.scheduler == None:
        return None

    return scheduler

In [ ]:
model = build_model()
optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
scheduler = fetch_scheduler(optimizer)

Wandb используется для стороннего отслеживания процесса обучения. Поскольку удаленные среды вроде google colab могут отрубать среду выполнения при ошибке/завершении обучения с потерей локальных данных (Kaggle, вроде, таким не особо страдает), настоятельно рекомендую подключить аккаунт wandb и раскомментить строки с запуском wandb и сохранением моделей, или же прописать в процесс обучения сохранение данных о модели в другое место, например, на Google Disk.

In [ ]:
#Здесь предполагается, что у вас в Kaggle в Secrets (во вкладке Add-ons) прописан api-key для Wandb
#В противном случае, он "подключается" к нему (на деле - нет).

import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("WANDB")
    wandb.login(key=api_key)
    anonymous = None
except:
    anonymous = "must"
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

In [ ]:
#Ручной вход в wandb, если у вас есть там аккаунт и есть api-key
#api_key = 'длинный_ключ_из_профиля_wandb'
#try:
#    wandb.login(key=api_key)
#    anonymous = None
#except:
#    anonymous = "must"

Начало обучения. Переименуйте seg_model_name на свое усмотрение. Если используете wandb - раскоментируйте строки внутри функций, отвечающих за обучение (там прописано логирование данных и сохранение моделей) и поменяйте параметры wandb.init.

Обратите внимание, что тяжелые модели (вроде UNet++ с b6 backbone) требуют много видеопамяти на видеокарте, и при запуске локально на обычных не топовых игровых видеокартах CUDA может вылететь с ошибкой "out of memory". На Kaggle или Google colab используются видеокарты большим объемом видеопамяти (например, Google Colab и Kaggle используют NVidia Tesla T4 с 16ГБ видеопамяти), поэтому при появлении данной ошибки рекомендую запустить модель в удаленной среде выполнения.

P.S. Хотя на небольших датасетах все обучалось норм, при прогоне на всем датасете с Unet++ b1 backbone у меня loss на train уходил в отрицательные значения (на valid все ок было), так что тут возможна в каком-то месте ошибка. По идее, loss не должен быть отрицательным.

In [ ]:
for fold in range(1):
    seg_model_name = 'UNetPPEffb1_LAST'
    print(f'#'*15)
    print(f'### Fold: {fold}')
    print(f'#'*15)
    run = wandb.init(project='med-segmentation-unetppb1',
                     config={k:v for k, v in dict(vars(CFG)).items() if '__' not in k},
                     anonymous=anonymous,
                     name=f"fold-{fold}|dim-{CFG.img_size[0]}x{CFG.img_size[1]}|model-{CFG.model_name}",
                     group=CFG.comment,
                    )
    train_loader, valid_loader = train_loader,valid_loader
    model     =   build_model()
    optimizer = optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
    scheduler =  fetch_scheduler(optimizer)

    model, historys_UNET = run_training(model, optimizer, scheduler,
                                device=CFG.device,
                                num_epochs=CFG.epochs)
    # run.finish()
    # display(ipd.IFrame(run.url, width=1000, height=720))
    plt.figure(figsize=(12,9))

    plt.plot(historys_UNET['Train Loss'], label=f'Training Loss')
    plt.plot(historys_UNET['Valid Loss'], label=f'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(f'Loss History-model{seg_model_name}')


    plt.figure(figsize=(17,9))
    plt.subplot(1, 2, 1)
    plt.plot(historys_UNET['Train Dice'], label=f'Training Dice')
    plt.plot(historys_UNET['Valid Dice'], label=f'Validation Dice')
    plt.xlabel('Epoch')
    plt.ylabel('Dice')
    plt.legend()
    plt.title(f'Dice History-model{seg_model_name}')

    plt.subplot(1, 2, 2)
    plt.plot(historys_UNET['Train Jaccard'], label=f'Training Jaccard')
    plt.plot(historys_UNET['Valid Jaccard'], label=f'Validation Jaccard')
    plt.xlabel('Epoch')
    plt.ylabel('Jaccard')
    plt.legend()
    plt.title(f'Jaccard History-model{seg_model_name}')
    plt.show()
    torch.save(model.state_dict(), f'{seg_model_name}_std.pth')
    torch.save(model, f'{seg_model_name}.pt')

In [ ]:
history = historys_UNET
print(f""" MODEL: {seg_model_name} Train DICE MAX:{max(history['Train Dice'])} IOU MAX:{max(history['Train Jaccard'])} Train Loss MIN:{min(history['Train Loss'])}
        MODEL: {seg_model_name} VAL DICE MAX:{max(history['Valid Dice'])} VAL IOU MAX:{max(history['Valid Jaccard'])} VAL Loss MIN:{min(history['Valid Loss'])} \n """)

In [ ]:
!zip model_weights.zip *.pth *.bin  # Архивируйте все веса

In [ ]:
from IPython.display import FileLink
FileLink('model_weights.zip')  # Ссылка для скачивания

Очистите кэш GPU перед обучением новой модели

In [ ]:
gpus = GPUtil.getGPUs()
GPUtil.showUtilization()
for i in range(len(gpus)):
    gpu = gpus[i]
    free_memory = gpu.memoryFree
torch.cuda.empty_cache()

# Валидация

Очистка кэша GPU

In [ ]:
gpus = GPUtil.getGPUs()
GPUtil.showUtilization()
for i in range(len(gpus)):
    gpu = gpus[i]
    free_memory = gpu.memoryFree
torch.cuda.empty_cache()

In [ ]:
valid_dataset = BuildDataset(x_val, y_val, transforms=data_transforms['valid'])
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, pin_memory=True)

Загружаем веса модели. В weight_c1 укажите название файла с весами необходимой модели. В параметрах model укажите тот encoder_name, с которым обучались веса модели.

In [ ]:
weight_c1 = 'UNetPPEffb1_LAST_std.pth'
model = smp.UnetPlusPlus(
    encoder_name="efficientnet-b1", encoder_weights=None, in_channels=1, classes=1)
model.to('cuda')
model.load_state_dict(torch.load(
    weight_c1, map_location='cuda'))
model.eval()

In [ ]:
device='cpu'

In [ ]:
preds = []
for step, (images, masks) in enumerate(valid_loader):
        images  = images.to(device, dtype=torch.float).cpu()
        masks   = masks.to(device, dtype=torch.float).cpu()
        models=model.cpu()

        for fold in range(1):
            # model = load_model(f"best_epoch-{fold:02d}.bin")
            with torch.no_grad():
                pred = models(images).cpu()
                pred = (nn.Sigmoid()(pred)>0.5).double()

                #preds = torch.argmax(nn.Sigmoid()(pred),axis=1).double()
                #val_dice = dice_coef(mask, pred).cpu().detach().numpy()
                #val_jaccard = iou_coef(mask, pred).cpu().detach().numpy()
                #val_scores.append([val_dice, val_jaccard])


                print( sum(sum(sum(sum(pred)))))
            preds.append(pred)

        images  = images.cpu().detach()
        preds = torch.mean(torch.stack(preds, dim=0), dim=0).cpu().detach()
        break

In [ ]:
#Альтернативный расчет Dice

def dice_coef_2(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    smooth = 0.0001
    intersection = (y_true*y_pred).sum()
    print(intersection)
    print(((y_true_f).sum() + (y_pred_f).sum() + smooth))
    print((y_true_f).sum())
    print((y_pred_f).sum())
    return (2. * intersection + smooth) / ((y_true_f).sum() + (y_pred_f).sum() + smooth)

In [ ]:
#Метрика Dice-Sørensen coefficient

dice_coef(masks, preds).cpu().detach().numpy()

In [ ]:
#Метрика IoU (Jaccard index)

iou_coef(masks, preds).cpu().detach().numpy()

Результаты работы модели: изображение, изображение + размеченая маска сегментации, изображение + предсказанная маска сегментации.

Здесь можно визуально оценить, насколько хорошо (или плохо) ваша модель работает.

In [ ]:
plot_batch(images, images, size=5)
plot_batch(images, masks, size=5)
plot_batch(images, preds, size=5)